In [1]:
import os, sys
import tensorflow as tf
os.chdir('..')
join = os.path.join(os.getcwd(), '_global')
sys.path.extend([join])
from _global.config import *
from _global.funcs import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tensorflow.keras import *
import albumentations as A
from PIL import Image
import cv2
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
def predict_for_image(bytes, model):
    image_bytes, _, img_shape = decode_img(bytes, resize=True)
    img_tensor = tf.cast(image_bytes, tf.uint8)
    img_tensor_expanded = tf.expand_dims(img_tensor, axis=0)

    bbxs = model.predict(img_tensor_expanded)
    bbxs = bbxs[0][0][0]
    print(bbxs[:20])

    threshold = 1
    # bbxs = [bbxs[x:x+4] for x in range(0, len(bbxs), 4) if bbxs[x] != 0 and bbxs[x+1] != 0]

    bbxs_real = []
    c = 0
    prev_coordinates = [sys.maxsize, sys.maxsize, sys.maxsize, sys.maxsize]
    for _ in range(MAX_BBXS * 4):
        c+=1
        if c % 4 != 0:
            continue
        curr = c-1
        if bbxs[curr] < 0 or bbxs[curr-1] <0 or bbxs[curr-2]<0 or bbxs[curr-3]<0:
            continue
        
        if (abs(bbxs[curr-3] - prev_coordinates[0]) < threshold or
            abs(bbxs[curr-2] - prev_coordinates[1]) < threshold):
            continue

        temp = [bbxs[curr-3], bbxs[curr-2], bbxs[curr-1], bbxs[curr]] 
        bbxs_real.append(temp)
        prev_coordinates = temp[:]

    return bbxs_real, img_tensor, img_shape
    # img = mpimg.imread(path)
    # plt.imshow(img)
def predict_for_image_with_path(path, model):
    with tf.io.gfile.GFile(path ,'rb') as file:
        return predict_for_image(file.read(), model)

In [3]:
def draw_bbx(img, bbx, color=(255, 0, 0), thickness=1):
    p1 = int(bbx[0]), int(bbx[1])
    p2 = int(bbx[0] + bbx[2]), int(bbx[1] + bbx[3])
    
    cv2.rectangle(img, p1, p2,color, thickness)

def draw_bbxs_on_image(img_tensor, bbxs, img_shape=(*IMAGE_SIZES, 3), bbxs_scaled_down=True, window_name="default"):
    if not bbxs_scaled_down:
        x_ratio = img_shape / IMAGE_SIZES[0]
        y_ratio = img_shape / IMAGE_SIZES[1]

        for bbx in bbxs:

            bbx[0] *= x_ratio
            bbx[2] *= x_ratio

            bbx[1] *= y_ratio
            bbx[3] *= y_ratio


    img = img_tensor.numpy()

    for bbx in bbxs:
        draw_bbx(img, bbx)

    cv2.imshow(window_name, img)
    cv2.moveWindow(window_name, -1920, 0)
    cv2.setWindowProperty(window_name, cv2.WND_PROP_TOPMOST, 1)

    while cv2.getWindowProperty(window_name, 0) >= 0:
        k = cv2.waitKey(0)
        if k == 27:
            break
        continue
    
    cv2.destroyAllWindows()

In [4]:
def draw_bbxs_from_tfrecord(tfrecord_path, record_number):
    if not isinstance(tfrecord_path, list):
        tfrecord_path = [tfrecord_path]
    
    dataset = tf.data.TFRecordDataset(tfrecord_path)

    for i in range(len(dataset)):
        if i +1 != record_number:
            continue
        
        curr = dataset[i][0]
        image, bbxs = parse_function(curr)

        draw_bbxs_on_image(image, bbxs, 'from_tfrecord')

In [5]:
# losses.custom_loss = loss_fn_practical
model = tf.keras.models.load_model(OUTPUT_MODEL, custom_objects={'loss_fn_practical': loss_fn_practical})

In [6]:
# [67.77223   47.06517   16.923975  25.885923  50.304874  35.963966
#   6.2854676 10.204706  41.274918  30.414097   4.0224295  7.105852
#  37.16802   27.614225   3.1189811  5.787781  33.005714  24.588493
#   2.2111108  4.368665 ]

# [71.112305  49.371758  17.722971  27.159712  52.783493  37.700424
#   6.570285  10.685776  43.320473  31.924595   4.2354403  7.480593
#  38.9796    28.967466   3.2910812  6.0731983 34.608833  25.760878
#   2.3060017  4.58374  ]

In [7]:
path = os.path.join(TEST_IMAGES, "49--Greeting\\49_Greeting_peoplegreeting_49_2.jpg")
path = os.path.join(TRAIN_IMAGES, "1--Handshaking\\1_Handshaking_Handshaking_1_82.jpg")
bbxs, img_tensor, shape = predict_for_image_with_path(path, model)
draw_bbxs_on_image(img_tensor, bbxs)

[77.57479   49.142696  12.00176   19.49504   42.44342    0.
  4.2657413  7.570258   0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.       ]
